In [1]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from components import Components
import os
load_dotenv()

e:\Github Repositories\RAG-FYP2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

#### Setting up OpenAI GPT4 Mini

In [2]:
token = os.getenv("GITHUB_TOKEN")
route = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"


In [3]:
client = OpenAI(
    base_url=route,
    api_key=token,
)

In [4]:
def get_response(prompt):
    response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """Instructions:
                        - Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'
                        - Be concise and to the point.""",
        },
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=model_name,
    temperature=0.1,
    max_tokens=1024,
    top_p=1.
)
    return response.choices[0].message.content

#### Setting up GroqLLM


In [2]:
rag_components = Components("Snowflake/snowflake-arctic-embed-l","","llama3-8b-8192")

In [3]:
llm = rag_components.get_groq_llm(system_prompt=""" Instructions:
        - Try your hardest to answer the question correctly.
        - Keep it short and concise.""")

#### Loading Data

In [4]:
df_qa = pd.read_csv('../data/MainDataset/data/QA.csv')

In [5]:
question_list = []
for questions in df_qa['question']:
    question_list.append(questions)

In [7]:
import time
response_list = []
for i in range(len(question_list)): 
    time.sleep(5)
    print(question_list[i])
    #response = get_response(question_list[i])
    response = llm.complete(question_list[i])
    print(response)
    rowIndex = df_qa.index[i]
    #response_list.append(response)
    df_qa.at[rowIndex, 'gen_answer'] = response.text
    print('')
    print("--"*100)
    print(f'{i}/{128}')
 


what was the climate like in the cretaceous period
During the Cretaceous period, which spanned from approximately 145 to 65 million years ago, the climate was generally warmer and more humid than it is today. Here are some key features of the Cretaceous climate:

1. High CO2 levels: Atmospheric CO2 concentrations were significantly higher than they are today, likely around 1,000-2,000 parts per million (ppm), compared to around 400 ppm today. This led to a stronger greenhouse effect, trapping more heat and contributing to the warmer climate.
2. No polar ice caps: There were no permanent polar ice caps during the Cretaceous period. The poles were likely covered in ice sheets that melted and reformed seasonally, but there was no permanent ice cover.
3. Higher sea levels: The Cretaceous period saw higher sea levels due to the warmer climate and the presence of more water in the oceans. This led to the formation of many coastal and marine ecosystems.
4. More extreme weather events: The Cre

In [8]:
df_qa

,question,answers,gen_answer
0,what was the climate like in the cretaceous pe...,Warmer,"During the Cretaceous period, which spanned fr..."
1,who first used a touch screen in a smartphone,IBM Simon,The first smartphone with a touch screen was t...
2,how do the digestive system and endocrine syst...,The digestive and endocrine systems work toget...,The digestive system and endocrine system work...
3,what does leukemia do to the body,It harms the body's ability to make healthy bl...,Leukemia is a type of cancer that affects the ...
4,meaning of the word attitude,It is a way of feeling or acting toward a pers...,"The word ""attitude"" refers to a person's menta..."
...,...,...,...
123,How is oxygen produced from carbon dioxide on ...,"['ultraviolet radiation', 'ultraviolet radiati...",Oxygen is not produced from carbon dioxide on ...
124,What was the Anglo-Norman language's final form?,"['Modern English', 'Modern English', 'Modern E...","The Anglo-Norman language, also known as Norma..."
125,A function problem is an example of what?,"['a computational problem', 'a computational p...",A function problem is an example of a type of ...
126,How many outputs are expected for each input i...,"['a single output', 'single', 'single']","In a function problem, the expected output for..."


In [9]:
df_qa.to_csv(f'../data/MainDataset/results/Official/llama-8b-8192.csv',index=False)